<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/weekly_SocialDistancing_UrbanRuralCode_Income_GDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic: Load and preprocess County level Social Distancing Data and Income/GDP Data

In [417]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [418]:
!pip install wget

## Import Libraries

In [419]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture
import datetime

## Mount Google drive


In [420]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [421]:
from google.colab import drive
# drive.mount('/content/drive')

In [422]:
!ls /content/drive/My\ Drive/MasterProject-Personal/data

ls: cannot access '/content/drive/My Drive/MasterProject-Personal/data': No such file or directory


# Data Load

## 1 Load County Population


In [423]:
county_population_US = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv',low_memory=False)
print(county_population_US.shape)

(3195, 4)


In [424]:
wget.download('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')
county_population_US = pd.read_csv('covid_county_population_usafacts.csv',low_memory=False)
print(county_population_US.shape)




(3195, 4)


In [425]:
county_population_US.head(2)

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


## 2 Load Covid-19 case details (until July 12)

In [426]:
!ls '/content/drive/My Drive/MasterProject-Personal/data/'

ls: cannot access '/content/drive/My Drive/MasterProject-Personal/data/': No such file or directory


### Note about data:
John Hopkins university updates data every day hence we are pulling from repository directly

**US Confirmed url** :https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv

**US deaths url**: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv

In [427]:
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv']

[wget.download(url) for url in urls]        

['time_series_covid19_confirmed_US (5).csv',
 'time_series_covid19_deaths_US (5).csv']

In [428]:
confirmed_US = pd.read_csv('time_series_covid19_confirmed_US.csv',low_memory=False)
death_US = pd.read_csv('time_series_covid19_deaths_US.csv',low_memory=False)
print(confirmed_US.shape)
print(death_US.shape)
print(confirmed_US.head(2))
death_US.head(2)

(3340, 206)
(3340, 207)
   UID iso2 iso3  code3  FIPS  ... 7/30/20 7/31/20 8/1/20  8/2/20  8/3/20
0   16   AS  ASM     16  60.0  ...       0       0      0       0       0
1  316   GU  GUM    316  66.0  ...     356     356    356     368     375

[2 rows x 206 columns]


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,...,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",55641,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",164229,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


In [429]:
#print(len(mask_data['state_name'].unique()))
print(len(confirmed_US['Province_State'].unique()))
confirmed_US['Province_State'].unique()

58


array(['American Samoa', 'Guam', 'Northern Mariana Islands',
       'Puerto Rico', 'Virgin Islands', 'Alabama', 'Alaska', 'Arizona',
       'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Diamond Princess',
       'Grand Princess'], dtype=object)

## 3 .Load Non-Pharmaceutical Interventions (NPIs) county level data

NPIs Include:
* SDO - Social Distancing of vulnerable persons;
* SD - Social Distancing of the general population;
* GS_XX - Gathering Size limitations;
* CPV - Closure of Public Venues;
* PC - Closure of schools and universities;
* NESC - Non-Essential Services Closure;
* LD - Lock Down (pending);

Link: 
1. https://www.keystonestrategy.com/coronavirus-covid19-intervention-dataset-model/
2. https://github.com/Keystone-Strategy/covid19-intervention-data
3. https://medium.com/@henrike.steimer/what-works-against-the-spread-of-covid-19-c0ecd4888386

In [430]:
import wget
wget.download('https://raw.githubusercontent.com/Keystone-Strategy/covid19-intervention-data/master/complete_npis_inherited_policies.csv')




'complete_npis_inherited_policies (5).csv'

In [431]:
data_NPI = pd.read_csv('complete_npis_inherited_policies.csv',low_memory=False)
print(data_NPI.shape)

(7389, 10)


In [432]:
data_NPI.tail(5)

,fips,county,state,npi,start_date,end_date,citation,note,re-opening_note,re-opening_citation
7384,24510,Baltimore,Maryland,non-essential_services_closure,3/23/2020,5/15/2020,https://governor.maryland.gov/wp-content/uploa...,retail stores can open at 50% capacity,NaN,NaN
7385,24510,Baltimore,Maryland,religious_gatherings_banned,3/19/2020,5/15/2020,https://governor.maryland.gov/wpcontent/upload...,implied based on gather of 10 max order made e...,NaN,NaN
7386,24510,Baltimore,Maryland,school closure,3/16/2020,NaN,https://news.maryland.gov/msde/state-superinte...,schools closed for remainder of school year,NaN,NaN
7387,24510,Baltimore,Maryland,shelter_in_place,3/30/2020,5/15/2020,https://governor.maryland.gov/2020/03/30/ascov...,certain groups advised to still stay at home b...,NaN,NaN
7388,24510,Baltimore,Maryland,social_distancing,NaN,NaN,NaN,NaN,NaN,NaN


## 4 Load US_County-level_Summaries

* **counties.csv**: contains demographic, socioeconomic, health care, education and transit data for each county in the 50 states and Washington DC. Note that this also includes analogous data for states, as available.

* **interventions.csv**: contains the dates that counties (or states governing them) took measures to mitigate the spread by restricting gatherings, given as the proleptic Gregorian ordinal of the date, where January 1 of year 1 has t = 1. 
It has now been updated to include intervention rollbacks. Any type of restaurant or gym reopening was taken as the rollback date (ex. the county could have reopened at 25% capacity and only outdoor, all the way to business as usual)

Link:
https://github.com/JieYingWu/COVID-19_US_County-level_Summaries/tree/master/data

In [433]:
wget.download('https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/counties.csv')
data_county_summary = pd.read_csv('counties.csv',low_memory=False)
print(data_county_summary.shape)

(3273, 347)


In [434]:
data_county_summary.dtypes

FIPS                                 int64
State                               object
Area_Name                           object
Rural-urban_Continuum Code_2013    float64
Urban_Influence_Code_2013          float64
                                    ...   
Number of AGGRAVATED ASSAULTS      float64
BURGLRY                            float64
LARCENY                            float64
MOTOR VEHICLE THEFTS               float64
ARSON                              float64
Length: 347, dtype: object

In [435]:
data_county_summary.head(1)

,FIPS,State,Area_Name,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,POP_ESTIMATE_2018,N_POP_CHG_2018,Births_2018,Deaths_2018,NATURAL_INC_2018,INTERNATIONAL_MIG_2018,DOMESTIC_MIG_2018,NET_MIG_2018,RESIDUAL_2018,GQ_ESTIMATES_2018,R_birth_2018,R_death_2018,R_NATURAL_INC_2018,R_INTERNATIONAL_MIG_2018,R_DOMESTIC_MIG_2018,R_NET_MIG_2018,Less than a high school diploma 2014-18,High school diploma only 2014-18,Some college or associate's degree 2014-18,Bachelor's degree or higher 2014-18,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a high school diploma only 2014-18,Percent of adults completing some college or associate's degree 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,PCTPOVALL_2018,CI90LBALLP_2018,CI90UBALLP_2018,POV017_2018,CI90LB017_2018,CI90UB017_2018,PCTPOV017_2018,...,Urology (AAMC),Vascular & Interventional Radiology (AAMC),Vascular Surgery (AAMC),Total nurse practitioners (2019),Total physician assistants (2019),Total Hospitals (2019),Internal Medicine Primary Care (2019),Family Medicine/General Practice Primary Care (2019),Pediatrics Primary Care (2019),Obstetrics & Gynecology Primary Care (2019),Geriatrics Primary Care (2019),Total Primary Care Physicians (2019),Psychiatry specialists (2019),Surgery specialists (2019),Anesthesiology specialists (2019),Emergency Medicine specialists (2019),Radiology specialists (2019),Cardiology specialists (2019),Oncology (Cancer) specialists (2019),Endocrinology Diabetes and Metabolism specialists (2019),All Other Specialties specialists (2019),Total Specialist Physicians (2019),ICU Beds,transit_scores - population weighted averages aggregated from town/city level to county,crime_rate_per_100000,COUNTY POPULATION-AGENCIES REPORT ARRESTS,COUNTY POPULATION-AGENCIES REPORT CRIMES,NUMBER OF AGENCIES IN COUNTY REPORT ARRESTS,NUMBER OF AGENCIES IN COUNTY REPORT CRIMES,COVERAGE INDICATOR,Total number of UCR (Uniform Crime Report) Index crimes excluding arson.,Total number of UCR (Uniform Crime Report) index crimes reported including arson,MURDER,RAPE,ROBBERY,Number of AGGRAVATED ASSAULTS,BURGLRY,LARCENY,MOTOR VEHICLE THEFTS,ARSON
0,0,US,United States,NaN,NaN,NaN,327167434,2020313.0,3855500.0,2814013.0,1041487.0,978826.0,0.0,978826.0,0.0,8091609.0,NaN,NaN,NaN,NaN,NaN,NaN,26948057,59265308,63365655,68867051,12.3,27.1,29.0,31.5,41852315.0,41619366.0,42085264.0,13.1,13.0,13.2,12997532.0,12873127.0,13121937.0,18.0,...,NaN,NaN,NaN,165480.0,88604.0,5198.0,197506.0,139407.0,87637.0,53913.0,1393.0,479856.0,54935.0,53002.0,50121.0,55671.0,47828.0,32640.0,20473.0,8046.0,202723.0,525439.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [436]:
data_county_summary = data_county_summary[data_county_summary['State']=='CA']
data_county_summary[data_county_summary['State']=='CA'].isnull().sum()

FIPS                               0
State                              0
Area_Name                          0
Rural-urban_Continuum Code_2013    1
Urban_Influence_Code_2013          1
                                  ..
Number of AGGRAVATED ASSAULTS      1
BURGLRY                            1
LARCENY                            1
MOTOR VEHICLE THEFTS               1
ARSON                              1
Length: 347, dtype: int64

In [437]:
data_county_summary.reset_index(drop=True)

,FIPS,State,Area_Name,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2013,Economic_typology_2015,POP_ESTIMATE_2018,N_POP_CHG_2018,Births_2018,Deaths_2018,NATURAL_INC_2018,INTERNATIONAL_MIG_2018,DOMESTIC_MIG_2018,NET_MIG_2018,RESIDUAL_2018,GQ_ESTIMATES_2018,R_birth_2018,R_death_2018,R_NATURAL_INC_2018,R_INTERNATIONAL_MIG_2018,R_DOMESTIC_MIG_2018,R_NET_MIG_2018,Less than a high school diploma 2014-18,High school diploma only 2014-18,Some college or associate's degree 2014-18,Bachelor's degree or higher 2014-18,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a high school diploma only 2014-18,Percent of adults completing some college or associate's degree 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,PCTPOVALL_2018,CI90LBALLP_2018,CI90UBALLP_2018,POV017_2018,CI90LB017_2018,CI90UB017_2018,PCTPOV017_2018,...,Urology (AAMC),Vascular & Interventional Radiology (AAMC),Vascular Surgery (AAMC),Total nurse practitioners (2019),Total physician assistants (2019),Total Hospitals (2019),Internal Medicine Primary Care (2019),Family Medicine/General Practice Primary Care (2019),Pediatrics Primary Care (2019),Obstetrics & Gynecology Primary Care (2019),Geriatrics Primary Care (2019),Total Primary Care Physicians (2019),Psychiatry specialists (2019),Surgery specialists (2019),Anesthesiology specialists (2019),Emergency Medicine specialists (2019),Radiology specialists (2019),Cardiology specialists (2019),Oncology (Cancer) specialists (2019),Endocrinology Diabetes and Metabolism specialists (2019),All Other Specialties specialists (2019),Total Specialist Physicians (2019),ICU Beds,transit_scores - population weighted averages aggregated from town/city level to county,crime_rate_per_100000,COUNTY POPULATION-AGENCIES REPORT ARRESTS,COUNTY POPULATION-AGENCIES REPORT CRIMES,NUMBER OF AGENCIES IN COUNTY REPORT ARRESTS,NUMBER OF AGENCIES IN COUNTY REPORT CRIMES,COVERAGE INDICATOR,Total number of UCR (Uniform Crime Report) Index crimes excluding arson.,Total number of UCR (Uniform Crime Report) index crimes reported including arson,MURDER,RAPE,ROBBERY,Number of AGGRAVATED ASSAULTS,BURGLRY,LARCENY,MOTOR VEHICLE THEFTS,ARSON
0,6000,CA,California,NaN,NaN,NaN,39557045,157696.0,477145.0,280674.0,196471.0,117797.0,-156068.0,-38271.0,-504.0,825850.0,12.1,7.1,5.0,3.0,-4.0,-1.0,4475565,5391120,7633463,8718737,17.1,20.6,29.1,33.3,4972955.0,4927111.0,5018799.0,12.8,12.7,12.9,1539628.0,1512838.0,1566418.0,17.4,...,1091.000000,399.000000,410.000000,15414.000000,7917.000000,359.000000,22363.000000,15198.000000,10500.000000,5956.000000,118.000000,54135.000000,7288.000000,5367.000000,6098.000000,6050.000000,5012.000000,3235.000000,1898.000000,822.000000,23001.000000,58771.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6001,CA,Alameda County,1.0,1.0,0.0,1666753,8622.0,19132.0,11150.0,7982.0,10717.0,-10035.0,682.0,-42.0,36417.0,11.5,6.7,4.8,6.4,-6.0,0.4,139234,203838,281631,531701,12.0,17.6,24.4,46.0,147394.0,137363.0,157425.0,9.0,8.4,9.6,33305.0,28853.0,37757.0,9.8,...,45.969751,16.812036,17.275525,649.475479,333.586179,15.126618,942.274564,640.374226,442.421988,250.958606,4.971980,2281.001365,307.082995,226.140839,256.941837,254.919336,211.182762,136.308108,79.973041,34.635321,969.156967,2476.341207,282.0,7.109445e+08,7.536936e+09,1544062.0,1544062.0,26.0,22.0,100.0,11900.0,62130.0,153.0,477.0,6220.0,5050.0,13245.0,35841.0,13044.0,337.0
2,6003,CA,Alpine County,8.0,4.0,5.0,1101,-20.0,8.0,6.0,2.0,0.0,-23.0,-23.0,1.0,24.0,7.2,5.4,1.8,0.0,-20.7,-20.7,87,225,237,235,11.1,28.7,30.2,30.0,190.0,143.0,237.0,17.3,13.0,21.6,66.0,48.0,84.0,34.7,...,0.030366,0.011105,0.011412,0.429021,0.220356,0.009992,0.622434,0.423009,0.292249,0.165775,0.003284,1.506751,0.202849,0.149381,0.169727,0.168391,0.139500,0.090040,0.052827,0.022879,0.640192,1.635786,0.0,6.858736e+07,6.902502e+09,1112.0,1112.0,2.0,2.0,100.0,8.0,43.0,0.0,3.0,0.0,5.0,18.0,20.0,5.0,0.

### County Intervention Data

In [438]:
wget.download('https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/interventions.csv')
data_county_interventions = pd.read_csv('interventions.csv',low_memory=False)
print(data_county_interventions.shape)

(3273, 16)


In [439]:
data_county_interventions=data_county_interventions[data_county_interventions['STATE']=='CA']
data_county_interventions.head(3)

,FIPS,STATE,AREA_NAME,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban,stay at home rollback,>50 gatherings rollback,>500 gatherings rollback,restaurant dine-in rollback,entertainment/gym rollback
191,6000,CA,california,737503.0,737503.0,737503.0,737503.0,737499.0,737499.0,737500.0,737495.0,737554.0,NaN,NaN,737554.0,737554.0
192,6001,CA,alameda county,737503.0,737503.0,737503.0,737503.0,737499.0,737499.0,737500.0,737495.0,737554.0,NaN,NaN,737594.0,737554.0
193,6003,CA,alpine county,737503.0,737503.0,737503.0,737503.0,737499.0,737499.0,737500.0,737495.0,737558.0,737558.0,NaN,737558.0,737558.0


In [440]:
import datetime

def date_change(x):
    date = datetime.date.fromordinal(x)
    #print(date.month, date.day, date.year)
    m= str(date.day)
    d= str(date.day)
    y= str(date.year)
    if int(m) < 10:
      m = '0'+str(m)
    if int(d) < 10:
      d = '0'+str(d)
  
    return m+'/'+d+'/'+y[-2::]

In [441]:
data_county_interventions['entertainment/gym']=data_county_interventions['entertainment/gym'].apply(lambda x: date_change(int(x)))
data_county_interventions['>50 gatherings']=data_county_interventions['>50 gatherings'].apply(lambda x: date_change(int(x)))
data_county_interventions['public schools']=data_county_interventions['public schools'].apply(lambda x: date_change(int(x)))
data_county_interventions['restaurant dine-in']=data_county_interventions['restaurant dine-in'].apply(lambda x: date_change(int(x)))


In [442]:
data_county_interventions.head(5)




,FIPS,STATE,AREA_NAME,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban,stay at home rollback,>50 gatherings rollback,>500 gatherings rollback,restaurant dine-in rollback,entertainment/gym rollback
191,6000,CA,california,737503.0,19/19/20,737503.0,19/19/20,15/15/20,15/15/20,737500.0,737495.0,737554.0,NaN,NaN,737554.0,737554.0
192,6001,CA,alameda county,737503.0,19/19/20,737503.0,19/19/20,15/15/20,15/15/20,737500.0,737495.0,737554.0,NaN,NaN,737594.0,737554.0
193,6003,CA,alpine county,737503.0,19/19/20,737503.0,19/19/20,15/15/20,15/15/20,737500.0,737495.0,737558.0,737558.0,NaN,737558.0,737558.0
194,6005,CA,amador county,737503.0,19/19/20,737503.0,19/19/20,15/15/20,15/15/20,737500.0,737495.0,737558.0,737558.0,NaN,737558.0,737558.0
195,6007,CA,butte county,737503.0,19/19/20,737503.0,19/19/20,15/15/20,15/15/20,737500.0,737495.0,737558.0,737558.0,NaN,737558.0,737558.0


## 5 Social Distancing Metrics

Link: 
1. https://docs.safegraph.com/docs/social-distancing-metrics#section-schema
2. https://www.safegraph.com/dashboard/covid19-shelter-in-place?s=US&d=07-25-2020&t=counties&m=index
3. https://docs.google.com/spreadsheets/d/1UNWvPzkUTTlXBZ6M6iGhM_7sr8h-MxsZdE7iOszkAmk/edit#gid=0

## 6 Lockdown data- govt strict policy index from Oxford

- https://github.com/OxCGRT/covid-policy-tracker

In [443]:
url = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
wget.download(url)




'OxCGRT_latest (5).csv'

In [444]:
data_policy = pd.read_csv('OxCGRT_latest.csv',low_memory=False)
print(data_policy.shape)

(38843, 42)


In [445]:
data_policy.head(2)

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,E1_Income support,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,20200101,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,20200102,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [446]:
data_policy[data_policy['CountryCode'] == 'USA']

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,E1_Income support,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
36239,United States,USA,20200101,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
36240,United States,USA,20200102,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
36241,United States,USA,20200103,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
36242,United States,USA,20200104,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
36243,United States,USA,20200105,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36451,United States,USA,20200731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4495014.0,152070.0,NaN,68.98,NaN,68.57,NaN,68.91,NaN,70.08,NaN,62.5
36452,United States,USA,20200801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4562037.0,153314.0,NaN,68.98,NaN,68.57,NaN,68.91,NaN,70.08,NaN,62.5
36453,United States,USA,20200802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4620444.0,154447.0,NaN,68.98,NaN,68.57,NaN,68.91,NaN,70.08,NaN,62.5
36454,United States,USA,20200803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4667955.0,154860.0,NaN,68.98,NaN,68.57,NaN,68.91,NaN,70.08,NaN,62.5


## 7 Data for Mobility Changes in Response to COVID-19

Field description
* country_code: ISO 3166-1 alpha-2 code.
* admin_level: 0 for country, 1 for admin1, 2 for admin2 granularity.
* admin1: GeoNames ADM1 feature name for the first-order administrative  division, such as a state in the United States.
* admin2: GeoNames ADM2 feature name for the second-order administrative division, such as a county or borough in the United States.
* fips: FIPS code, a standard geographic identifier, to make it easier to combine this data with other data sets.
* samples: The number of samples observed in the specified region.
* m50: The median of the max-distance mobility for all samples in the specified region.
* m50_index: The percent of normal m50 in the region, with normal m50 defined during 2020-02-17 to 2020-03-07.

Source: https://github.com/descarteslabs/DL-COVID-19

Related Refs:
0. https://arxiv.org/pdf/2003.14228.pdf
1. https://blog.nikhaldimann.com/2020/04/11/lockdown-hack-interlude-correlating-mobility-stats-pandemic-growth/
2. https://github.com/nikhaldi/covid-notebooks/blob/master/Mobility%20vs%20COVID-19%20growth%20plot.ipynb


In [447]:
wget.download('https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv')



'DL-us-mobility-daterow (5).csv'

In [448]:
data_mobility_change = pd.read_csv('DL-us-mobility-daterow.csv',low_memory=False)
data_mobility_change.head(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
0,2020-03-01,US,1,Alabama,NaN,1.0,133826,8.331,79
1,2020-03-02,US,1,Alabama,NaN,1.0,143632,10.398,98


In [449]:
data_mobility_change[(data_mobility_change['admin1']=='California') & (data_mobility_change['admin_level']== 2)]

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
25092,2020-03-01,US,2,California,Alameda County,6001.0,16737,3.684,58
25093,2020-03-02,US,2,California,Alameda County,6001.0,17269,6.182,98
25094,2020-03-03,US,2,California,Alameda County,6001.0,18181,6.271,100
25095,2020-03-04,US,2,California,Alameda County,6001.0,18495,6.209,99
25096,2020-03-05,US,2,California,Alameda County,6001.0,17496,6.654,106
...,...,...,...,...,...,...,...,...,...
33225,2020-07-28,US,2,California,Yuba County,6115.0,1368,5.144,54
33226,2020-07-29,US,2,California,Yuba County,6115.0,1364,4.360,46
33227,2020-07-30,US,2,California,Yuba County,6115.0,1399,4.844,51
33228,2020-07-31,US,2,California,Yuba County,6115.0,1375,4.555,48


In [450]:
filter_mobility_data= data_mobility_change[(data_mobility_change['admin1']=='California') & (data_mobility_change['admin_level']== 2)]

In [451]:
filter_mobility_data.reset_index(drop=True)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
0,2020-03-01,US,2,California,Alameda County,6001.0,16737,3.684,58
1,2020-03-02,US,2,California,Alameda County,6001.0,17269,6.182,98
2,2020-03-03,US,2,California,Alameda County,6001.0,18181,6.271,100
3,2020-03-04,US,2,California,Alameda County,6001.0,18495,6.209,99
4,2020-03-05,US,2,California,Alameda County,6001.0,17496,6.654,106
...,...,...,...,...,...,...,...,...,...
8133,2020-07-28,US,2,California,Yuba County,6115.0,1368,5.144,54
8134,2020-07-29,US,2,California,Yuba County,6115.0,1364,4.360,46
8135,2020-07-30,US,2,California,Yuba County,6115.0,1399,4.844,51
8136,2020-07-31,US,2,California,Yuba County,6115.0,1375,4.555,48


In [452]:
print(filter_mobility_data.admin2.nunique())
filter_mobility_data.admin2.unique()

54


array(['Alameda County', 'Amador County', 'Butte County',
       'Calaveras County', 'Colusa County', 'Contra Costa County',
       'Del Norte County', 'El Dorado County', 'Fresno County',
       'Glenn County', 'Humboldt County', 'Imperial County',
       'Inyo County', 'Kern County', 'Kings County', 'Lake County',
       'Lassen County', 'Los Angeles County', 'Madera County',
       'Marin County', 'Mariposa County', 'Mendocino County',
       'Merced County', 'Mono County', 'Monterey County', 'Napa County',
       'Nevada County', 'Orange County', 'Placer County', 'Plumas County',
       'Riverside County', 'Sacramento County', 'San Benito County',
       'San Bernardino County', 'San Diego County',
       'San Francisco County', 'San Joaquin County',
       'San Luis Obispo County', 'San Mateo County',
       'Santa Barbara County', 'Santa Clara County', 'Santa Cruz County',
       'Shasta County', 'Siskiyou County', 'Solano County',
       'Sonoma County', 'Stanislaus County', 'Su

In [453]:
print(filter_mobility_data.date.nunique())
filter_mobility_data.date.unique()

152


array(['2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
       '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
       '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
       '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16',
       '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20',
       '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24',
       '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28',
       '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01',
       '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05',
       '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09',
       '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13',
       '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17',
       '2020-04-18', '2020-04-19', '2020-04-21', '2020-04-22',
       '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26',
       '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30',
       '2020-05-01', '2020-05-02', '2020-05-03', '2020-

## 8 The Rural-Urban characteristics of the county
The rural-urban characteristics of the county was obtained from the United States Department of Agriculture Economic Research Service, Rural-Urban Continuum Codes
Link:
https://www.ers.usda.gov/data-products/rural-urban-continuum-codes.aspx

In [454]:
location = "drive/Shared drives/the-mean-sqaures/the-mean-squares/Colab_Dataset/Dataset/"


In [455]:
data_ruralurban_county_code = pd.read_csv(location+'ruralurbancodes2013.csv',low_memory=False)
print(data_ruralurban_county_code.shape)

(3221, 6)


In [456]:
data_ruralurban_county_code = data_ruralurban_county_code[(data_ruralurban_county_code['State']=='CA')]
data_ruralurban_county_code.head(3)

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description
186,6001,CA,Alameda County,"1,510,271",1,Metro - Counties in metro areas of 1 million p...
187,6003,CA,Alpine County,"1,175",8,"Nonmetro - Completely rural or less than 2,500..."
188,6005,CA,Amador County,"38,091",6,"Nonmetro - Urban population of 2,500 to 19,999..."


# Social Distancing Mobility Data: Daily to Weekly

## Fix Date Format

In [483]:
def fixDate(x):
  arr = x.split('/')
  m = arr[0]
  d = arr[1]
  y = arr[2]

  if int(m) < 10:
    m = '0'+str(m)
  if int(d) < 10:
    d = '0'+str(d)
  return m+'/'+d+'/'+y

In [484]:
fixDate('1/22/20')

'01/22/20'

In [485]:
def fixDate2(x):
  arr = x.split('-')
  d = arr[2]
  m = arr[1]
  y = arr[0]

 
  return m+'/'+d+'/'+y[-2:]

In [486]:
fixDate2('2020-03-01')

'03/01/20'

In [487]:
#filter_mobility_data.date.unique()
filter_mobility_data['date'] = filter_mobility_data['date'].apply(lambda x: fixDate2(x))
filter_mobility_data.reset_index(drop=True)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
0,03/01/20,US,2,California,Alameda County,6001.0,16737,3.684,58
1,03/02/20,US,2,California,Alameda County,6001.0,17269,6.182,98
2,03/03/20,US,2,California,Alameda County,6001.0,18181,6.271,100
3,03/04/20,US,2,California,Alameda County,6001.0,18495,6.209,99
4,03/05/20,US,2,California,Alameda County,6001.0,17496,6.654,106
...,...,...,...,...,...,...,...,...,...
8133,07/28/20,US,2,California,Yuba County,6115.0,1368,5.144,54
8134,07/29/20,US,2,California,Yuba County,6115.0,1364,4.360,46
8135,07/30/20,US,2,California,Yuba County,6115.0,1399,4.844,51
8136,07/31/20,US,2,California,Yuba County,6115.0,1375,4.555,48


## Generate Date-Week

In [488]:
day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
minDate = filter_mobility_data['date'].min()
print(minDate)
maxDate = filter_mobility_data['date'].max()
print(maxDate)
dateList = filter_mobility_data['date'].unique()
print(len(dateList))

dateList.sort()
#print(dateList)

03/01/20
08/01/20
152


In [519]:
index = 0
count = 0
date_lst = []
while index < len(dateList):
  row = []
  j = 7 - day;
  # print(index,j)
  # print(dateList[index:index+j])
  weekRange = dateList[index:index+j]
  index = index+j
  day = 0
  count +=1
  row.append(count)
  row.append(weekRange.min())
  row.append(weekRange.max())
  
  #print(row)
  date_lst.append(row)
  # if count > 5:
  #   break

In [520]:
week_df = pd.DataFrame(date_lst,index=None,columns=['WeekNumber','startDate','endDate'])
week_df

,WeekNumber,startDate,endDate
0,1,03/01/20,03/07/20
1,2,03/08/20,03/14/20
2,3,03/15/20,03/21/20
3,4,03/22/20,03/28/20
4,5,03/29/20,04/04/20
5,6,04/05/20,04/11/20
6,7,04/12/20,04/18/20
7,8,04/19/20,04/26/20
8,9,04/27/20,05/03/20
9,10,05/04/20,05/10/20


## Daily to Weekly

In [530]:
def compute_daily_to_weekly_average(countyName):
  index = 0
  count = 0
  day = datetime.datetime.strptime(minDate, '%m/%d/%y').weekday()
  if day == 6:
    day =0
  elif day < 6:
    day += 1
  
  while index < len(dateList):
    row = []
    j = 7 - day;
    if index+j >= len(dateList):
      break
    # print(dateList[index:index+j])
    #print(j)
    weekRange = dateList[index:index+j]
  
    #row.append(count)
    
    
    df_filter = filter_mobility_data[(filter_mobility_data['admin2'] == countyName) & (filter_mobility_data['date'] >= dateList[index]) & (filter_mobility_data['date'] < dateList[index+j] )]
    #getting empty for some county-need to check
    if not df_filter.empty:
        row.append(count)
        row.append(weekRange.min())
        row.append(weekRange.max())
        #print("weekRange.min()=",weekRange.min())
        #print("weekRange.max()=",weekRange.max())
        #print(df_filter)
        #print("count=",count)
        #print("############")
        row.append(df_filter['admin2'].tolist()[0])
        row.append(df_filter['fips'].tolist()[0])
        row.append(df_filter['samples'].sum())
        row.append(df_filter['m50'].mean())
        row.append(df_filter['m50_index'].mean())

        #print(row)
        lst.append(row)
    index = index+j
    day = 0
    count +=1
  return lst

In [531]:
county_names = filter_mobility_data['admin2'].unique()
print(len(county_names))

lst = []
for countyName in county_names:
  #print("countyName",countyName)
  compute_daily_to_weekly_average(countyName)

54


## Final data - Merge with Urban-Rural Code

In [532]:
df_final = pd.DataFrame(lst,index=None,columns=['WeekNumber','startDate','endDate','county_name','FIPS',
                                                'weekly_samples','weekly_m_50',
                                               'weekly_m_50_index'])

print(df_final.shape)
df_final.head(2)

(1130, 8)


,WeekNumber,startDate,endDate,county_name,FIPS,weekly_samples,weekly_m_50,weekly_m_50_index
0,0,03/01/20,03/07/20,Alameda County,6001.0,122036,5.796143,92.000000
1,1,03/08/20,03/14/20,Alameda County,6001.0,120619,4.768000,75.428571


In [533]:
df_final.to_csv(location+'Clean_DescartesLabsMobilityData_byCounty.csv', index=False)

In [534]:
df_socialdistance_urcode =  pd.merge(df_final, data_ruralurban_county_code,  how='inner', left_on=['county_name'], right_on = ['County_Name'])

In [535]:
df_socialdistance_urcode.to_csv(location+'Clean_socialdistance_urcode_byCounty.csv',index=False)